In [1]:
from libs.spark_session import Spark
from libs.utils import *
from libs.ingestors import Ingestor

In [2]:
spark = Spark().spark

In [3]:
catalog = 'bronze'
database = 'postgres'
table = 'financeiro.clientes'
data_format = 'parquet'
id_field = 'cliente_id'
timestamp_field = 'modificado_em'


In [4]:
ingestor = Ingestor(
    spark,
    catalog,
    database,
    table,
    data_format
)

In [5]:
if table_exists(spark,catalog,ingestor.table_formatted):
    print(f"ingesting table {ingestor.table_name}")
    ingestor.execute()

ingesting table bronze.postgres_financeiro_clientes


In [6]:
spark.sql(f"select count(*) from {ingestor.table_name}").show()

+--------+
|count(1)|
+--------+
|   13600|
+--------+

